<a href="https://colab.research.google.com/github/JContro/pytorch_projects/blob/master/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np


In [0]:
data_dir = '/content/drive/My Drive/Data/flowers'

In [0]:
import h5py 
import os, sys

In [0]:
class dataset_h5(torch.utils.data.Dataset):
    def __init__(self, in_file, transform=None):
        super(dataset_h5, self).__init__()
 
        self.file = h5py.File(in_file, 'r')
        self.transform = transform
 
    def __getitem__(self, index):
        x = torch.from_numpy(self.file['X_train'][index, :].transpose(2,0,1)).float()
        y = np.argmax(self.file['y_train'][index, :])
        
        # Preprocessing each image
        if self.transform is not None:
            x = self.transform(x)        
        
        return (x, y)
 
    def __len__(self):
        return self.file['X_train'].shape[0]

In [0]:
transform = None

In [0]:
# Dev set
input_file = h5py.File("/content/drive/My Drive/Data/flowers.h5", 'r')
X_dev = torch.from_numpy(input_file['X_dev'][:].transpose(0,3,1,2)).float()
y_dev = torch.from_numpy(np.argmax(np.squeeze(input_file['y_dev']), axis=1))

In [0]:
X_dev = X_dev.cuda()
y_dev = y_dev.cuda()

In [0]:
dataset = dataset_h5("/content/drive/My Drive/Data/flowers.h5",transform=transform)
dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=128, shuffle=True,
        drop_last=True)

In [131]:
# Run this to test your data loader
images, labels = next(iter(dataloader))
images

tensor([[[[209., 209., 210.,  ..., 231., 231., 230.],
          [210., 210., 211.,  ..., 232., 231., 230.],
          [210., 210., 211.,  ..., 232., 231., 231.],
          ...,
          [  0.,   2.,   3.,  ...,  57.,  62.,  31.],
          [  0.,   0.,   0.,  ...,  62.,  47.,  64.],
          [  0.,   0.,   0.,  ...,  56.,  80.,  49.]],

         [[152., 152., 154.,  ..., 171., 171., 170.],
          [153., 153., 154.,  ..., 173., 173., 172.],
          [153., 153., 154.,  ..., 174., 174., 174.],
          ...,
          [137., 143., 146.,  ..., 255., 255., 144.],
          [144., 146., 148.,  ..., 250., 253., 251.],
          [147., 149., 150.,  ..., 254., 247., 252.]],

         [[ 96.,  96.,  96.,  ..., 117., 117., 116.],
          [ 97.,  97.,  98.,  ..., 117., 115., 113.],
          [ 98.,  98.,  98.,  ..., 115., 113., 113.],
          ...,
          [151., 156., 161.,  ..., 255., 255., 151.],
          [159., 162., 164.,  ..., 249., 249., 251.],
          [163., 165., 165.,  ...

In [0]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 12, kernel_size=10, stride=3, padding=0),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(12),
            nn.ReLU(inplace=True),
            
            # Defining another 2D convolution layer
            nn.Conv2d(12, 24, kernel_size=5, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(24),
            nn.ReLU(inplace=True),
            

            nn.Conv2d(24, 48, kernel_size=5, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True),
            
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(3072, 500),
            nn.Linear(500, 500),
            nn.Linear(500,63),
            nn.Linear(63, 5)

        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        # print(x.shape)
        x = x.reshape(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [115]:
# defining the model
model = Net()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 12, kernel_size=(10, 10), stride=(3, 3))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Conv2d(24, 48, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=3072, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=500, bia

In [0]:
# del model
# torch.cuda.empty_cache()

In [0]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [62]:
params

830507

In [0]:
def train(epoch):
    model.train()
    tr_loss = 0
    train_losses = []
    # getting the training set
    c = 0

    for images, labels in dataloader:
      c += 1
    
      # converting the data into GPU format
      if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()
          

      # clearing the Gradients of the model parameters
      optimizer.zero_grad()
    
      # prediction for training and validation set
      output = model(images)
      print(output)
      break
      # computing the training and validation loss
      loss_train = criterion(output, labels)
      train_losses.append(loss_train)
      

      # computing the updated weights of all the model parameters
      loss_train.backward()
      optimizer.step()
      tr_loss = loss_train.item()
      
    print(f"The epoch {epoch + 1} has a loss of {tr_loss}")

    # Evauate accuracy
    model.eval()
    dev_output = model(X_dev)
    print(dev_output)
    # print(dev_output.cpu().detach().numpy())
    # print()
    model.train()

In [128]:
epochs = 1
for e in range(epochs):
  train(e)

tensor([[ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,   6.0946,  92.1177],
        [ 68.7246, 105.2242,  28.2701,  